# Convolutional Neural Networks for Visual Recognition
http://cs231n.github.io

## Image Classification

### Nearest Neighbor Classifier

** $L_1$ distance**

$d_1(I_1,I_2) = \sum_{p}|I_1^p-I_2^p|$

where $I_1$, $I_2$ are vectors of two images being compared.

**$L_2$ distance**

$d_2(I_1,I_2) = \sqrt{\sum_{p}(I_1^p-I_2^p)^2}$

**k-Nearest Neighbor Classifier**

Motivation: instead of finding single closest image in the training set, we find the top k closest images and have them vote on the label of the test image.


![alt text](./img/k-nearest.png "Title")

### Pros and Cons of Nearest Neighbor Classifier
**Pros**
* Simple
* Takes no time to train

**Cons**
* May work on low-dimensional data but distance over high-dimensional spaces can be very counter-intuitive
* Images that are nearby each other are much more a function of the general color distribution of the images, or the type of background rather than their semantic identity.
* The classifier must remember all the training data and store for future comparisons with the test data.
* Classifying a test image is expensive since it requires a comparison to all training images.


## Validation sets for Hyperparameter tuning

**Example of hyperparameters:** the setting for k of k-nearest neighbor classifier.

**Tuning hyperparameters:** split training test in two: a slightly smaller training set and what we call a validation set.

In case where the size of training data might be small, people sometimes use more sophisticated techniques for hyperparameter tuning called **cross-validation**.

![alt text](./img/k-fold.png "Title")

Classic way is to split your training data randomly into train/val splits. As a rule of thumb, between 70-90% of your data usually goes to the train split. 

## Linear Classification

The approach will have two major components: a **score function** that maps the raw data to class scores, and a **loss function** that quantifies the agreement between the predicted scores and the ground truth labels.

For example, in CIFAR-10 we have a training set of N = 50,000 images, each with D = 32 x 32 x 3 = 3072 pixels, and K = 10, since there are 10 distinct classes (dog, cat, car, etc). 

### Linear Classifier

$f(x_i, W, b) = Wx_i+b$

where $x_i$ contains all pixels in the i-th image flattened into a single [3072 x 1] column. W is often called weights and is [10 x 3072] and b is called bias vector, with size of [10 x 1].

**Example**

![alt text](./img/classifier.png "Title")

In the example shown above, the linear classifier compute the scores of a class as a weighted sum of all its pixel values across all 3 of its color channels. We assume the image only has 4 pixels and that we have 3 classes.


**Bias Trick**

We can combine two sets of parameters (the biases b and weights W) into a single matrix that holds both of them, in which we get:

$f(x_i, W) = Wx_i$


### Loss Function

**Multiclass Support Vector Machine (SVM)**

SVM loss is setup so that the correct class for each image to have a score higher than the incorrect class by some fixed amount margin $\Delta$. Notice that it’s sometimes helpful to anthropomorphise the loss functions as we did above: The SVM “wants” a certain outcome in the sense that the outcome would yield a lower loss (which is good).

$$L_i = \sum_{j \neq y_i}\max (0, s_j-s_{yi}+\Delta)$$

where $y_i$ is the index of the correct class and $s_j = f(x_i,W)_j$ (the score for j-th class is the j-th element). 

We can also rewrite the loss function as:

$$L_i = \sum_{j \neq y_i}\max (0, w_j^T-w_{yi}^T+\Delta)$$


### Regularization
With the loss function presented above, a potential problem would be that there are a set of parameters W correctly classify every example. We want to encode some preference for certain set of weights W over others to remove this ambiguity.

We can do so by extending the loss function with a regularization penalty R(w). The most common regularization penalty is L2 norm.

$$R(W) = \sum_{k} \sum_{l} W_{k,l}^2$$

The full multiclass SVM loss becomes:
$$L = \frac{1}{N} \sum_i L_i + \lambda R(W)$$

or expanding this out in its full form:

$$L = \frac{1}{N} \sum_i \sum_{j \neq y_i}max(0, w_j^T-w_{yi}^T+\Delta) + \lambda \sum_{k} \sum_{l} W_{k,l}^2$$

L2 penalty leads to the appealing max margin property in SVM. The most appealing property is that penalizing large weights tends to improve generalization, because it means that no input dimension can have a very large influence on the scores by itself.

 ### Practical Consideration
 
 **Setting Delta**
 
 It turns out that this hyperparameter can safely be set to $\Delta$=1.0
 
 **Relation to Binary Support Vector Machine**
 
 $$L_i = Cmax(0,1,-y_iw^Tx_i)+R(W)$$
 
 where C is a hyperparameter and $y_i \in \{-1,1\}$. This can be regard as a special case when there are only two classes in this SVM.

### Softmax Classifier

It turns out that SVM is one of two commonly seen classifiers. The other popular choice is Softmax classifier.

$$L_i = -\log (\frac{e^{f_{yi}}}{\sum_je^{f_j}})$$ or equivalently $$L_i = -f_{yi}+\log \sum_je^{f_j}$$

It takes a vector of arbitrary real-valued scores and squashes it to vector values between 0 and one that sum to one.

**Information theory view**

The cross-entropy between a "true" distribution p and an estimated distributed q is defined as 

$$H(p,1)=-\sum_xp(x)l\log q(x)$$

The softmax classifier is hence minimizing cross-entropy between the estimated class probabilities and the "true" distribution.

Moreover, since the cross-entropy can be written in terms of entropy and the Kullback-Leibler divergence as $H(p,q)=H(p)+D_{KL}(p||q)$ and the entropy of the data function p is zero, this is also equivalent to minimizing the KL divergence between the two distributions.

In other words, the cross-entropy objective wants the predicted distribution to have all of its mass on the correct answer.

**Probabilistic interpretation**

$$P(y_i|x_i:W)=\frac{e^{f_{yi}}} {\sum_j e^{f_j}}$$

can be interpreted as the probability assigned to the correct label $y_i$, given the image $x_i$ and parameterized by W.

**Practical issues: Numeric stability**

When we are writing code for computing the Softmax function in practice, the intermediate term $e^{f_{yi}}$ and $\sum_j e^{f_j}$ may be very large due to exponential. So it is important to use a normalization trick.

$$\frac{e^{f_{yi}}} {\sum_j e^{f_j}} = \frac{Ce^{f_{yi}}} {\sum_j e^{f_j}}=\frac{e^{f_{yi}+\log C}} {\sum_j e^{{f_j}+\log C}}$$

A common choice for C is to set $\log C = -\max_jf_j$.

**Possibly confusing**

To be precise, the SVM classifier uses the hinge loss, or also sometimes called the max-margin loss. The Softmax classifier uses cross-entropy loss.

### SVM vs. Softmax

In both cases we computer the same score vector f. The difference is in the interpretation of the scores in f: The SVM interprets these as class cores adn its loss function encourage the correct class to have a score higher by a margin than the other class scores.

The Softmax classifier instead interprets the score as (unnormalized) log probabilities for each class and then encourages the (normalized) log probability of the correct class to be high. 

In practices, SVM and Softmax are usually comparable.

## Optimization: Stochastic Gradient Descent

Optimization: the process of finding the set of parameters W that minimize the loss function.

Assume a simple dataset that contains three 1-dimensional points and three classes. The full SVM loss (without regulation) becomes:

$$L_0 = \max(0,w_1^Tx_0-w_0^Tx_0+1)+\max(0,w_2^Tx_0-w_0^Tx_0+1)$$
$$L_1 = \max(0,w_0^Tx_1-w_1^Tx_1+1)+\max(0,w_2^Tx_1-w_1^Tx_1+1)$$
$$L_2 = \max(0,w_0^Tx_2-w_2^Tx_2+1)+\max(0,w_1^Tx_2-w_2^Tx_2+1)$$
$$L=(L_0+L_1+L2)/3$$

![alt text](./img/sum.png "Title")

The SVM cost funciton is an example of a convex function.


### Optimization Strategy

**Random Search**

Not quite a good idea..


In [ ]:
bestloss = float("inf") # Python assigns the highest possible float value
for num in xrange(1000):
  W = np.random.randn(10, 3073) * 0.0001 # generate random parameters
  loss = L(X_train, Y_train, W) # get the loss over the entire training set
  if loss < bestloss: # keep track of the best solution
    bestloss = loss
    bestW = W
  print 'in attempt %d the loss was %f, best %f' % (num, loss, bestloss)

**Random Local Search**

Extend one foot in a random direction and take a step only if it leads downhill. Concretely, we will start out with a random W, generate random perturbation $\delta$W to it and if the loss at perturbed W+$\delta$W is lower, we will perform an update

**Following the Gradient**

The gradient is a generalization of slope for functions that don't take a signle number but a vector of numbers. Gradient is just a vector of slopes (more commonly referred as derivatives) for each dimension in the input space.

The mathematical expression for the derivative of a 1-D function with respect to its input is:

$$\frac{df(x)}{dx}=lim_{h \to 0} \frac{f(x+h)-f(x)}{h}$$

When the functions of interest take a vector of numbers instead of a single number, we call the derivatives partial derivatives and the gradient is simply the vector or partial derivatives in each dimension.

### Computing the gradient numerically with finite differences

In [1]:
def eval_numerical_gradient(f, x):
  """ 
  a naive implementation of numerical gradient of f at x 
  - f should be a function that takes a single argument
  - x is the point (numpy array) to evaluate the gradient at
  """ 

  fx = f(x) # evaluate function value at original point
  grad = np.zeros(x.shape)
  h = 0.00001

  # iterate over all indexes in x
  it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
  while not it.finished:

    # evaluate function at x+h
    ix = it.multi_index
    old_value = x[ix]
    x[ix] = old_value + h # increment by h
    fxh = f(x) # evalute f(x + h)
    x[ix] = old_value # restore to previous value (very important!)

    # compute the partial derivative
    grad[ix] = (fxh - fx) / h # the slope
    it.iternext() # step to next dimension

  return grad

**Effect of step size**

The gradient tells us the direction in which the function has the steepest rate of increase, but it does not tell us how far along this direction we should step. As we will see later in the course, choosing the step size (h) will become one of the most important hyperparameter settings in training a neural network. 

**Computing the gradient analytically with Calculus**

The numerical gradient is very simple to compute using the finite difference approximation but the downside is that it is approximate and that is very computationally expensive to compute. The second way to compute the gradient is analytically using Calculus, which allows us to derive a direct formula for the gradient that is also very fast to compute. However, unlike the numerical gradient, it can be more error prone to implement, which is why in practice it is very common to compute the analytic gradient and compare it to the numerical gradient to check correctness of your implementation. This is called **gradient check**.

With SVM loss function for a single data point:
$$L_i = \sum_{j \neq y_i}\max (0, s_j-s_{yi}+\Delta)$$

We can differentiate the function with respect to the weights.

$$\nabla w_{yi}L_i = - (\sum_{j\neq y_i}𝟙(w_j^Tx_i-w_{yi}^Tx_i)+\Delta > 0))x_i$$

𝟙 is the indicator function that is one if the condition inside is true or zero otherwise. 

For the other rows where $j\neq y_i$, the gradient is:

$$\nabla w_{j}L_i = - (\sum_{j\neq y_i}𝟙(w_j^Tx_i-w_{yi}^Tx_i)+\Delta > 0))x_i$$


### Gradient Descent

Now that we can compute the gradient of the loss function, the procedure of repeatedly evaluating the gradient and then performing a parameter update is called Gradient Descent.

**Mini-batch gradient descent**

In large-scale applications, it is wasteful to compute full loss function over the entire datasets in order to perform a single parameter update. Avery common approach to address this challenge is to computer gradient over batches of training data.

The extreme case of this is a setting where mini-batch contains only a single example. The process is called **Stochastic Gradient Descent (SGD)**. Even though SGD technically refers to using a single example at a time to evaluate the gradient, you will hear people use the term SGD even when referring to mini-batch gradient descent 

![alt text](./img/summary.png "Title")

## Backpropagation, Intuitions

### Simple expression and interpretation of the gradient

$$f(x,y)=xy$$
$$\frac{\partial f}{\partial x} = y$$
$$\frac{\partial f}{\partial y} = x$$


Derivative indicates the rate of change of a function with respect to variable surrounding an infinitesimally small region near a particular point:

$$\frac{df(x)}{dx}=\lim_{h \to 0} \frac{f(x+h)-f(x)}{h}$$

The derivative of each variable tells you the sensitivity of the whole expression on its value. If we have $\frac{\partial f}{\partial y} = 4$, we expect that increasing value of y by a some very small amount h would increase the output of the function by 4h.

The derivative on each variable tells you the sensitivity of the whole expression on its value.